In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)
block_size = 8
batch_size = 4
max_iters = 100000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
# device = 'cpu'

## Open file wizard_of_oz.txt

In [20]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
# text

In [21]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [22]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
# string_to_int

In [23]:
int_to_string = { i:ch for i,ch in enumerate(chars) }
# int_to_string

In [24]:
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# en = encode("Zulqar")
# en
# de = decode(en)
# de
# encode(text)


In [25]:
data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [26]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
print("data = " , data , len(data))
print("train_data = " , train_data , len(train_data))
print("val_data = " , val_data , len(val_data))

data =  tensor([80,  1,  1,  ..., 29, 67, 57]) 232309
train_data =  tensor([80,  1,  1,  ..., 71,  0, 69]) 185847
val_data =  tensor([65, 54, 73,  ..., 29, 67, 57]) 46462


In [27]:
data = torch.tensor([5, 6, 7, 8, 9, 10, 11, 12, 13, 14])  # len = 10
block_size = 4
batch_size = 3
ix = torch.randint(10 - 8, (6,))
ix

tensor([1, 0, 1, 0, 1, 1])

In [28]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[67, 57,  1, 76],
        [59, 68, 71, 66],
        [ 1, 61, 54, 57]], device='cuda:0')
targets:
tensor([[57,  1, 76, 58],
        [68, 71, 66, 58],
        [61, 54, 57,  1]], device='cuda:0')


In [29]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [30]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        # print("self.token_embedding_table =>" , self.token_embedding_table)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        # print("<============== forward() ==============> ")
        # print("logits => " , logits.shape)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

            # print("B, T, C " , B, T, C)
            # print("logits " , logits.shape)
            # print("targets => " , targets.shape)
            # print("loss => " , loss)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        print("<============== generate() ==============> ")
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)

            
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)

            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
            

            # print("index ---> [embedding matrix] ---> logits => " , logits.shape )
            # # print("loss => " , loss) 
            # print("last time step -> logits => " , logits.shape)
            # print("logits ---> [softmax] ---> probs => " , probs.shape)
            # print("max prob sample -> index_next => " , index_next.shape)
            # print("index => " , index.shape)
        return index

In [31]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)
print("m => " , m)
context = torch.zeros((1,1), dtype=torch.long, device=device)
print("context => " , context)
generated_chars = decode(m.generate(context, max_new_tokens=50)[0].tolist())

m =>  BigramLanguageModel(
  (token_embedding_table): Embedding(81, 81)
)
context =>  tensor([[0]], device='cuda:0')
<============== generate() ==============> 


In [32]:

print(generated_chars)


y6vS;JzK (cyFG!i9;m-AkwE(oSuzbpx7*SUM)Mki'BDLmAFvz


In [33]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        print("\n=============================================================================")
        print(f"iter ============================> {iter} / {max_iters}")
        print("===============================================================================\n")
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


iter ============================> 0 / 100000

step: 0, train loss: 4.838, val loss: 4.833

iter ============================> 250 / 100000

step: 250, train loss: 4.812, val loss: 4.784

iter ============================> 500 / 100000

step: 500, train loss: 4.750, val loss: 4.773

iter ============================> 750 / 100000

step: 750, train loss: 4.714, val loss: 4.723

iter ============================> 1000 / 100000

step: 1000, train loss: 4.651, val loss: 4.647

iter ============================> 1250 / 100000

step: 1250, train loss: 4.585, val loss: 4.581

iter ============================> 1500 / 100000

step: 1500, train loss: 4.555, val loss: 4.545

iter ============================> 1750 / 100000

step: 1750, train loss: 4.516, val loss: 4.537

iter ============================> 2000 / 100000

step: 2000, train loss: 4.491, val loss: 4.480

iter ============================> 2250 / 100000

step: 2250, train loss: 4.436, val loss: 4.442

iter ==========================

***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim

In [34]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

<============== generate() ==============> 

t
"

y M_Q?"

"airy. bll alof m spelareniknged f se caclidin're s;  b. unereanizle, garon an theleoroy quro cad
THVso!"
Thow hecuitimat alk wono of treever trantheat n, a qurowo b, w Vpely. y Winanoun herexd sthton nd ld clou por he skean t cithan; an shofre. t in, s thizale s to cted j

an rno rotan  ne ad t, abo lansofonthe ske m, VRZ60(Do sisstheppatorspluerils iongr blyoat tugoere hathend Thealawofaincearsmarene

bot omytowearsond t fad ckes f theratherke plywatore awith, I ere, eked hy.
ton


In [35]:
250%250

0